In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pytesseract
import cv2
import re

# Step 1: Load and Prepare Dataset
df = pd.read_csv('haram_data.csv')  # Replace with your dataset path
df = df.dropna(subset=['text', 'label'])  # Drop rows with missing values
df['label'] = df['label'].map({'halal': 0, 'haram': 1})  # Map labels to 0 (halal) and 1 (haram)

# Step 2: Train-Test Split
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Vectorize Text Data
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 4: Train the Model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Step 5: Evaluate the Model
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.99      0.99     58501
           1       0.99      0.97      0.98     47118

    accuracy                           0.98    105619
   macro avg       0.99      0.98      0.98    105619
weighted avg       0.98      0.98      0.98    105619



In [14]:
# Step 6: Define Function to Classify Ingredients from Text
def classify_ingredients(text):
    ingredients = re.split(r'[,\n]', text)  # Split by commas or new lines
    results = []
    for ingredient in ingredients:
        ingredient_cleaned = ingredient.strip().lower()
        if ingredient_cleaned:
            vectorized_ingredient = vectorizer.transform([ingredient_cleaned])
            prediction = model.predict(vectorized_ingredient)
            results.append((ingredient_cleaned, 'Haram' if prediction[0] == 1 else 'Halal'))
    return results

# Step 7: Use OCR to Extract Ingredients from Image
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(img_gray)
    return text

# Step 8: Full Pipeline for Image Upload and Classification
def halal_check(image_path):
    text = extract_text_from_image(image_path)
    print("Extracted Text:", text)
    classification_results = classify_ingredients(text)
    for ingredient, status in classification_results:
        print(f"Ingredient: {ingredient}, Status: {status}")

In [18]:
# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\USER\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'


In [19]:
# Example Usage
image_path = 'haram1.png'  # Replace with your image path
halal_check(image_path)


Extracted Text: Pate

Ingredients CON R F

Pork jowl (without skin) 15 15 15

Pork shoulder 55 35 35

Pork liver 10 10 10

Broth 18 18 18

Roe - 20 -

Fish meat - - 20
Sodium chloride 13
Black pepper 0.15
Herbal pepper 0.25
Marjoram 0.15
Nutmeg 0.15

Ingredient: pate, Status: Halal
Ingredient: ingredients con r f, Status: Halal
Ingredient: pork jowl (without skin) 15 15 15, Status: Haram
Ingredient: pork shoulder 55 35 35, Status: Haram
Ingredient: pork liver 10 10 10, Status: Haram
Ingredient: broth 18 18 18, Status: Halal
Ingredient: roe - 20 -, Status: Halal
Ingredient: fish meat - - 20, Status: Halal
Ingredient: sodium chloride 13, Status: Halal
Ingredient: black pepper 0.15, Status: Halal
Ingredient: herbal pepper 0.25, Status: Halal
Ingredient: marjoram 0.15, Status: Halal
Ingredient: nutmeg 0.15, Status: Halal
